In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.own import OwnSingleModel
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import plot_model

from env import *

%matplotlib inline

C:\ProgramFiles\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')

MODEL_FILE = os.path.join(SAVED_MODELS_PATH, 'own', 'single.h5')
RESULT_CSV = os.path.join(RESULTS_PATH, 'own_single.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

seq = 'words_seq'
train_x = np.array(list(train_with_seq[seq]))
val_x = np.array(list(val_with_seq[seq]))
test_x = np.array(list(test_with_seq[seq]))

y_cols = LABELS
val_ys = []
train_outputs = []
val_outputs = []
for col in y_cols:
    train_y = train_with_seq[col] + 2
    val_y = val_with_seq[col] + 2
    val_ys.append(val_y)
    train_y_onehot = to_categorical(train_y)
    val_y_onehot = to_categorical(val_y)
    train_outputs.append(train_y_onehot)
    val_outputs.append(val_y_onehot)

In [6]:
model = OwnSingleModel(max_len = train_x.shape[1], embedding = embedding)
model._model.summary()
plot_model(model._model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 843)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 843, 100)     5000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 841, 32)      9632        embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 839, 32)      16032       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [7]:
model.fit(train_x, train_outputs, 
          validation_data = (val_x, val_outputs),
          model_file = MODEL_FILE
         )

Epoch 1/300
105000/105000 [==============================] - 553s 5ms/step - loss: 12.7462 - dense_1_loss: 0.3350 - dense_2_loss: 0.4715 - dense_3_loss: 0.4196 - dense_4_loss: 0.4012 - dense_5_loss: 0.6905 - dense_6_loss: 0.2590 - dense_7_loss: 0.4118 - dense_8_loss: 0.9298 - dense_9_loss: 0.5348 - dense_10_loss: 0.6726 - dense_11_loss: 0.6813 - dense_12_loss: 0.6653 - dense_13_loss: 0.7812 - dense_14_loss: 0.6663 - dense_15_loss: 0.9443 - dense_16_loss: 0.8147 - dense_17_loss: 0.7541 - dense_18_loss: 0.8931 - dense_19_loss: 0.7599 - dense_20_loss: 0.6601

Epoch 00001: _f1_monitor improved from -inf to 0.55220, saving model to C:\Users\LINAN\Documents\Projects\ai_challenger_sentiment\data\saved_models\own\single.h5


C:\ProgramFiles\anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,dense_1_loss,dense_2_loss,dense_3_loss,dense_4_loss,dense_5_loss,dense_6_loss,dense_7_loss,dense_8_loss,dense_9_loss,dense_10_loss,dense_11_loss,dense_12_loss,dense_13_loss,dense_14_loss,dense_15_loss,dense_16_loss,dense_17_loss,dense_18_loss,dense_19_loss,dense_20_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\ProgramFiles\anaconda3\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,dense_1_loss,dense_2_loss,dense_3_loss,dense_4_loss,dense_5_loss,dense_6_loss,dense_7_loss,dense_8_loss,dense_9_loss,dense_10_loss,dense_11_loss,dense_12_loss,dense_13_loss,dense_14_loss,dense_15_loss,dense_16_loss,dense_17_loss,dense_18_loss,dense_19_loss,dense_20_loss,lr
  (self.monitor, '

Epoch 2/300
105000/105000 [==============================] - 546s 5ms/step - loss: 9.0011 - dense_1_loss: 0.2222 - dense_2_loss: 0.3456 - dense_3_loss: 0.3084 - dense_4_loss: 0.2996 - dense_5_loss: 0.4958 - dense_6_loss: 0.0959 - dense_7_loss: 0.2794 - dense_8_loss: 0.6730 - dense_9_loss: 0.3877 - dense_10_loss: 0.4856 - dense_11_loss: 0.4964 - dense_12_loss: 0.4700 - dense_13_loss: 0.5793 - dense_14_loss: 0.4855 - dense_15_loss: 0.7269 - dense_16_loss: 0.6174 - dense_17_loss: 0.5937 - dense_18_loss: 0.3764 - dense_19_loss: 0.5749 - dense_20_loss: 0.4873


C:\ProgramFiles\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00002: _f1_monitor improved from 0.55220 to 0.57749, saving model to C:\Users\LINAN\Documents\Projects\ai_challenger_sentiment\data\saved_models\own\single.h5
Epoch 3/300
105000/105000 [==============================] - 546s 5ms/step - loss: 8.1517 - dense_1_loss: 0.1917 - dense_2_loss: 0.3163 - dense_3_loss: 0.2748 - dense_4_loss: 0.2696 - dense_5_loss: 0.4455 - dense_6_loss: 0.0741 - dense_7_loss: 0.2401 - dense_8_loss: 0.6127 - dense_9_loss: 0.3503 - dense_10_loss: 0.4427 - dense_11_loss: 0.4506 - dense_12_loss: 0.4205 - dense_13_loss: 0.5322 - dense_14_loss: 0.4371 - dense_15_loss: 0.6727 - dense_16_loss: 0.5695 - dense_17_loss: 0.5540 - dense_18_loss: 0.3288 - dense_19_loss: 0.5305 - dense_20_loss: 0.4381

Epoch 00003: _f1_monitor improved from 0.57749 to 0.58204, saving model to C:\Users\LINAN\Documents\Projects\ai_challenger_sentiment\data\saved_models\own\single.h5
Epoch 4/300
105000/105000 [==============================] - 546s 5ms/step - loss: 7.6359 - dense_1_loss: 0

KeyboardInterrupt: 

In [8]:
model._model.load_weights(MODEL_FILE)
val_probs = model.predict(val_x)
test_probs = model.predict(test_x)

val_preds = list(map(lambda x: np.argmax(x, axis = -1), val_probs))
# val_preds = np.argmax(val_probs, axis = 1)
# test_preds = list(map(lambda x: np.argmax(x, axis = -1), test_probs))

In [9]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_ys, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.5593708492447703
The 1th f1: 0.43034597854106893
The 2th f1: 0.6235464992902724
The 3th f1: 0.5330423047229336
The 4th f1: 0.6890419807082055
The 5th f1: 0.62801814563896
The 6th f1: 0.5941150088812555
The 7th f1: 0.6610598964060818
The 8th f1: 0.5845026865025243
The 9th f1: 0.5922163949695857
The 10th f1: 0.5674573668384438
The 11th f1: 0.6131141758116125
The 12th f1: 0.6334922789371844
The 13th f1: 0.6021969096335009
The 14th f1: 0.5722941614966017
The 15th f1: 0.6280492200536627
The 16th f1: 0.4532371444557949
The 17th f1: 0.6239541086237658
The 18th f1: 0.5147784951764068
The 19th f1: 0.5840675014513939
The average f1 of val is 0.5843950553692012
